In [2]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.optimizers import SGD

In [7]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('mbi_questions_english.json').read())

In [8]:
words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Convert to lowercase before tokenizing
        wordList = nltk.word_tokenize(pattern.lower())
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Convert words to lowercase and lemmatize, ignoring specified characters
words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

# Save words and classes for future use
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Prepare training data
training = []
outputEmpty = [0] * len(classes)

In [9]:
for document in documents:
    bag = []
    wordPatterns = document[0]
    # Convert each word to lowercase and lemmatize
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

# Split data into training inputs and outputs
trainX = training[:, :len(words)]
trainY = training[:, len(words):]

# Build and compile the model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

# Define optimizer and compile model
optimizer = SGD(learning_rate=0.02, momentum=0.9, nesterov=True)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
hist = model.fit(np.array(trainX), np.array(trainY), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
11/11 [==============================] - 1s 2ms/step - loss: 2.9025 - accuracy: 0.0588
Epoch 2/200
11/11 [==============================] - 0s 2ms/step - loss: 2.7221 - accuracy: 0.1961
Epoch 3/200
11/11 [==============================] - 0s 2ms/step - loss: 2.5306 - accuracy: 0.1961
Epoch 4/200
11/11 [==============================] - 0s 2ms/step - loss: 2.2154 - accuracy: 0.2745
Epoch 5/200
11/11 [==============================] - 0s 2ms/step - loss: 2.2330 - accuracy: 0.3333
Epoch 6/200
11/11 [==============================] - 0s 2ms/step - loss: 1.8472 - accuracy: 0.5098
Epoch 7/200
11/11 [==============================] - 0s 2ms/step - loss: 1.6259 - accuracy: 0.5098
Epoch 8/200
11/11 [==============================] - 0s 2ms/step - loss: 1.4918 - accuracy: 0.5098
Epoch 9/200
11/11 [==============================] - 0s 1ms/step - loss: 1.2584 - accuracy: 0.5882
Epoch 10/200
11/11 [==============================] - 0s 2ms/step - loss: 1.1991 - accuracy: 0.6078
Epoch 11/

11/11 [==============================] - 0s 2ms/step - loss: 0.1428 - accuracy: 0.9608
Epoch 84/200
11/11 [==============================] - 0s 2ms/step - loss: 0.2270 - accuracy: 0.9412
Epoch 85/200
11/11 [==============================] - 0s 2ms/step - loss: 0.1741 - accuracy: 0.9804
Epoch 86/200
11/11 [==============================] - 0s 2ms/step - loss: 0.1772 - accuracy: 0.9412
Epoch 87/200
11/11 [==============================] - 0s 2ms/step - loss: 0.3475 - accuracy: 0.9216
Epoch 88/200
11/11 [==============================] - 0s 2ms/step - loss: 0.3411 - accuracy: 0.8824
Epoch 89/200
11/11 [==============================] - 0s 2ms/step - loss: 0.5820 - accuracy: 0.8235
Epoch 90/200
11/11 [==============================] - 0s 2ms/step - loss: 0.1314 - accuracy: 0.9804
Epoch 91/200
11/11 [==============================] - 0s 1ms/step - loss: 0.1795 - accuracy: 0.9412
Epoch 92/200
11/11 [==============================] - 0s 2ms/step - loss: 0.1174 - accuracy: 0.9804
Epoch 93/200


11/11 [==============================] - 0s 2ms/step - loss: 0.1376 - accuracy: 0.9216
Epoch 165/200
11/11 [==============================] - 0s 2ms/step - loss: 0.1689 - accuracy: 0.9608
Epoch 166/200
11/11 [==============================] - 0s 2ms/step - loss: 0.2731 - accuracy: 0.9608
Epoch 167/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0180 - accuracy: 1.0000
Epoch 168/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0134 - accuracy: 1.0000
Epoch 169/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0957 - accuracy: 0.9608
Epoch 170/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0559 - accuracy: 1.0000
Epoch 171/200
11/11 [==============================] - 0s 2ms/step - loss: 0.0331 - accuracy: 1.0000
Epoch 172/200
11/11 [==============================] - 0s 2ms/step - loss: 0.1271 - accuracy: 0.9608
Epoch 173/200
11/11 [==============================] - 0s 3ms/step - loss: 0.0238 - accuracy: 1.0000
Epoc

In [10]:
# Save the trained model
model.save('chatbot_model_eng.keras')
print('Done')

Done


In [11]:
import tensorflow as tf
print(tf.__version__)

2.13.0
